# Chapter 11 Exercise

In [1]:
# To support both python 2 and python 3
from __future__ import division, print_function, unicode_literals

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

# To plot pretty figures
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "deep"

def save_fig(fig_id, tight_layout=True):
    path = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID, fig_id + ".png")
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format='png', dpi=300)

In [2]:
import tensorflow as tf

(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
X_train = X_train.astype(np.float32).reshape(-1, 28*28) / 255.0
X_test = X_test.astype(np.float32).reshape(-1, 28*28) / 255.0
y_train = y_train.astype(np.int32)
y_test = y_test.astype(np.int32)
X_valid, X_train = X_train[:5000], X_train[5000:]
y_valid, y_train = y_train[:5000], y_train[5000:]

## Problem 8:

In [3]:
reset_graph()

he_init = tf.variance_scaling_initializer()

# Condensed form of the above code
def dnn(inputs,n_hidden_layers,n_per_h_layer,n_outputs,activation=tf.nn.elu,initializer=he_init,seed=42,learning_rate=0.01):
    # For Chapter 11 #8.1
    with tf.name_scope("dnn"):
        for layer_num in range(n_hidden_layers):
            inputs = tf.layers.dense(inputs, n_per_h_layer, 
                                     activation=activation, kernel_initializer=initializer, name="hidden%d" % (layer_num+1))
        outputs = inputs
        logits = tf.layers.dense(outputs, n_outputs, kernel_initializer=he_init, name="logits")
        y_prob = tf.nn.softmax(logits, name="y_prob")
    # For #8.2
    with tf.name_scope("loss"):
        xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
        loss = tf.reduce_mean(xentropy, name="loss")
    with tf.name_scope("train"):
        optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
        training_op = optimizer.minimize(loss, name="training_op")
        loss_summary = tf.summary.scalar('log_loss', loss)
    with tf.name_scope("eval"):
        correct = tf.nn.in_top_k(logits, y, 1)
        accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
    with tf.name_scope("init"):
        init = tf.global_variables_initializer()
    with tf.name_scope("save"):
        saver = tf.train.Saver()
    return y_prob, logits, loss, training_op, loss_summary, accuracy, init, saver

In [4]:
# For TensorBoard
from datetime import datetime

def log_dir(prefix=""):
    now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
    root_logdir = "tf_logs"
    if prefix:
        prefix += "-"
    name = prefix + "run-" + now
    return "{}/{}/".format(root_logdir, name)

In [5]:
# Graph for TensorBoard
# (not using an enhanced X, using X from before)
n_inputs = 28*28  # MNIST
n_hidden_layers = 5
n_per_h_layer = 100
n_outputs = 5
logdir = log_dir("dnn11")

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int32, shape=(None), name="y")
# inputs = X ???
y_prob, logits, loss, training_op, loss_summary, accuracy, init, saver = dnn(X,n_hidden_layers,n_per_h_layer,n_outputs)

file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

In [6]:
# For 8.2: 0 to 4
X_train1 = X_train[y_train < 5]
y_train1 = y_train[y_train < 5]
X_valid1 = X_valid[y_valid < 5]
y_valid1 = y_valid[y_valid < 5]
X_test1 = X_test[y_test < 5]
y_test1 = y_test[y_test < 5]

In [7]:
n_epochs = 1000
batch_size = 20

max_checks_without_progress = 20
checks_without_progress = 0
best_loss = np.infty

checkpoint_path = "/tmp/dnn11_0_to_4.ckpt"
checkpoint_epoch_path = checkpoint_path + ".epoch"
final_model_path = "./dnn11_0_to_4"

In [8]:
def shuffle_batch(X, y, batch_size):
    rnd_idx = np.random.permutation(len(X))
    n_batches = len(X) // batch_size
    for batch_idx in np.array_split(rnd_idx, n_batches):
        X_batch, y_batch = X[batch_idx], y[batch_idx]
        yield X_batch, y_batch

In [9]:
with tf.Session() as sess:
    if os.path.isfile(checkpoint_epoch_path):
        # if the checkpoint file exists, restore the model and load the epoch number
        with open(checkpoint_epoch_path, "rb") as f:
            start_epoch = int(f.read())
        print("Training was interrupted. Continuing at epoch", start_epoch)
        saver.restore(sess, checkpoint_path)
    else:
        start_epoch = 0
        sess.run(init)
        
    for epoch in range(n_epochs):
        for X_batch, y_batch in shuffle_batch(X_train1, y_train1, batch_size):
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        #acc_batch = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        #acc_val = accuracy.eval(feed_dict={X: X_valid1, y: y_valid1})
        loss_val, acc_val = sess.run([loss, accuracy], feed_dict={X: X_valid1, y: y_valid1})
        if loss_val < best_loss:
            save_path = saver.save(sess, "./my_mnist_model_0_to_4.ckpt")
            best_loss = loss_val
            checks_without_progress = 0
        else:
            checks_without_progress += 1
            if checks_without_progress > max_checks_without_progress:
                print("Early stopping!")
                break
        print("{}\tValidation loss: {:.6f}\tBest loss: {:.6f}\tAccuracy: {:.2f}%".format(
            epoch, loss_val, best_loss, acc_val * 100))

with tf.Session() as sess:
    saver.restore(sess, "./my_mnist_model_0_to_4.ckpt")
    acc_test = accuracy.eval(feed_dict={X: X_test1, y: y_test1})
    print("Final test accuracy: {:.2f}%".format(acc_test * 100))

    save_path = saver.save(sess, "./my_dnn11_0_to_4_final.ckpt")

0	Validation loss: 0.351242	Best loss: 0.351242	Accuracy: 90.58%
1	Validation loss: 0.111183	Best loss: 0.111183	Accuracy: 97.38%
2	Validation loss: 0.209247	Best loss: 0.111183	Accuracy: 96.68%
3	Validation loss: 0.105242	Best loss: 0.105242	Accuracy: 97.89%
4	Validation loss: 0.144679	Best loss: 0.105242	Accuracy: 97.58%
5	Validation loss: 0.399149	Best loss: 0.105242	Accuracy: 94.25%
6	Validation loss: 1.397254	Best loss: 0.105242	Accuracy: 50.74%
7	Validation loss: 0.451237	Best loss: 0.105242	Accuracy: 75.06%
8	Validation loss: 0.349017	Best loss: 0.105242	Accuracy: 93.39%
9	Validation loss: 0.344995	Best loss: 0.105242	Accuracy: 94.80%
10	Validation loss: 0.176267	Best loss: 0.105242	Accuracy: 97.38%
11	Validation loss: 0.239872	Best loss: 0.105242	Accuracy: 96.99%
12	Validation loss: 1.687223	Best loss: 0.105242	Accuracy: 22.01%
13	Validation loss: 1.617635	Best loss: 0.105242	Accuracy: 22.01%
14	Validation loss: 1.662950	Best loss: 0.105242	Accuracy: 20.91%
15	Validation loss: 

In [11]:
with tf.Session() as sess:
    saver.restore(sess, "./my_dnn11_0_to_4_final.ckpt") # or better, use save_path
    X_new_scaled = X_test1[:20]
    Z = logits.eval(feed_dict={X: X_new_scaled})
    y_pred = np.argmax(Z, axis=1)

INFO:tensorflow:Restoring parameters from ./my_dnn11_0_to_4_final.ckpt


In [12]:
from tensorflow_graph_in_jupyter import show_graph

show_graph(tf.get_default_graph())